## Plotting with the ArcGIS Python API
The ArcGIS Python API provides the "map widget" which improves on Foliums approach in that it has more of Python, less of a JavaScript interface. However, this API still seems to be under development and some aspects (especially these things called renderers) do appear to be more JavaScript than Python. Also, it appears only to work in the Jupyter Notebook setting, though it can be used to create ArcGIS Online Web Maps and also save static images. 

Nevertheless, it makes for a useful tool for vizualing spatial data. Below we examine a few quick examples, but the APS's documentation and examples are quite helpful in digging deeper into what it can do. 

https://developers.arcgis.com/python/guide/visualizing-data-with-the-spatially-enabled-dataframe/

###  Project setup
To start, we'll import the necessary packages and fetch datasets of NC HUC12s and some hog farm point locations within Duplin County. We'll also deal with spatial references among our datasets and then, to simplify the visualization, subset just the HUC12s containing hog farms... 
* Import packages

In [ ]:
#Import pandas for interacting with the data
import pandas as pd

#Import the API's components
from arcgis import GIS, GeoAccessor, GeoSeriesAccessor
from arcgis.features import SpatialDataFrame

#Connect to the GIS object
gis = GIS()

#Enable inline plotting. 
%matplotlib inline

* Fetch data to visualize

In [ ]:
#Get some data
sdfHUC12s = GeoAccessor.from_featureclass('./data/12Digit_HUC_Subwatersheds.shp')
sdfSites = GeoAccessor.from_featureclass('./data/DuplinSites_LL.shp')

* Manage spatial references (good to set all to same projection)

In [ ]:
#Check the spatial reference of the HUC12 dataset
sdfHUC12s.spatial.sr

In [ ]:
#Check the spatial reference of the Duplin sites dataset
sdfSites.spatial.sr

In [ ]:
#Reproject points to same sr as polygons
sdfSites.spatial.project(sdfHUC12s.spatial.sr)
sdfSites.spatial.sr

* Quick spatial analysis: subset just HUC12s containing the points

In [ ]:
#Select the HUCs intersecting the points
sdfHUCs_select = sdfHUC12s.spatial.relationship(sdfSites,op='contains')
sdfHUCs_select.shape

* Quick tabular analysis: classify values in the "Estimate" fields into low, med, and high categories

In [ ]:
#Set the class masks
low_mask = sdfSites['Estimated'] < 16
high_mask = sdfSites['Estimated'] > 26

In [ ]:
#Add a column to the SDF sites classifying "Estimate" values as low, med, high
sdfSites['Class'] = 'X'
sdfSites.loc[low_mask,'Class'] = 'low'
sdfSites.loc[(~low_mask) & (~high_mask),'Class'] = 'med'
sdfSites.loc[high_mask,'Class'] = 'high'

In [ ]:
#Show tallies of each class
sdfSites.Class.value_counts().plot(kind='bar');

### Simple plotting
Steps: 
* Create a map (optionally zoomed into a set region)
* Plot your spatial dataframe onto the map

In [ ]:
#Step1 : Create a map zoomed to Duplin Co then show it
map1 = gis.map('Duplin Co., NC')
map1

In [ ]:
#Step 2: Plot the points GeoAccessor object onto the map
sdfSites.spatial.plot(map_widget = map1)

In [ ]:
#Remove the previous plot and re-plot both the HUCs and the Points
map1.remove_layers()
sdfHUCs_select.spatial.plot(map1)
sdfSites.spatial.plot(map1)

### Controlling how features are displayed
Here, we'll map our points again, but this time using the symbolization example taken from the documentation:
* https://developers.arcgis.com/python/guide/visualizing-data-with-the-spatially-enabled-dataframe/#Plotting-the-DataFrame

Then, we'll play with the various values to get a feel for what they control. 

In [ ]:
map2 = gis.map('Duplin Co., NC')
map2

In [ ]:
#Taken directly from the API help example
sdfSites.spatial.plot(map_widget=map2,
                      symbol_type='simple',
                      symbol_style='d', # d - for diamonds
                      colors='Reds_r',
                      cstep=10,
                      outline_color='Blues',
                      marker_size=10)

Now, play with the values to see what they do
* Change the symbol type to crosses, then to squares
* Add a `line_width` attribute, setting it's value to 0.01
* Change the cstep to 0, then to 100, 200: what happens as the value gets larger?

In [ ]:
map2.remove_layers()
sdfSites.spatial.plot(map_widget=map2,
                      symbol_type='simple',
                      symbol_style='d', # d - for diamonds
                      colors='Reds_r',
                      cstep=10,
                      outline_color='Blues',
                      marker_size=10)

##### Renderers: 
https://developers.arcgis.com/python/guide/visualizing-data-with-the-spatially-enabled-dataframe/#Understanding-renderers<br>
https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#arcgis.features.GeoAccessor.plot<br>


In [ ]:
map3 = gis.map('Duplin Co., NC',zoomlevel=10)
map3

In [ ]:
map3.remove_layers()
sdfSites.spatial.plot(map_widget = map3,
                      renderer_type='h',
                      col='Class',
                      line_width=0.1,
                      alpha=0.4 
             )

* Change the symbol type from default circles to squares
* Change render types from simple to:
    * unique values on our "Class" attribute
    * class break values on our "Estimate" 

* Change the renderer type to 'h'
* Change the renderer type to 'c' (class) hten 